In [1]:
import pymssql  
conn = pymssql.connect(
    server='demosqlwesteurope.database.windows.net',
    user='student_01',
    password='!MiptSql_01',
    database='demo'
)

In [2]:
def q(query: str):
    cursor = conn.cursor()  
    cursor.execute(query) 
    row = cursor.fetchone()  
    while row:  
        for i, element in enumerate(row):
            print(str(row[i]), end="\t\t")
        print()
        row = cursor.fetchone()  

**1 -**

**2**

**3**

4

5

6

7

8\. no

9\. no

10\. no

11\. no

12\. no

13\. no

14\. no

15\. no

16\. no

17\. -

18. Сколько в среднем обслуживает клиентов менеджер филиала.

In [4]:
# TODO

19. Сколько всего клиентов обслужил филиал за определенный период.

In [5]:
# todo

20. Какой менеджер обслужил в филиале, максимальное кол-во клиентов

In [6]:
q("""--sql
with temp(managerId, checksSold) as (
    select salesManagerId, count(sales.checkId)
    from sales
    where salesManagerId is not Null
    group by salesManagerId
)

select managerId, surname, [names], checksSold
from temp
inner join salesManager on (salesManager.salesManagerId = temp.managerId)
where checksSold = (
    select max(checksSold)
    from temp
)
""")

82		Фирсова		Елена		78087		


21. Какой менеджер, принес максимальную выручку в филиале за определенный месяц

In [7]:
q("""SELECT max(salesRub) as maxSalesRub, salesManagerId
from sales 
where
    branchId = 4 and
    month(dateId) = 8
group by salesManagerId
""")

75093.42		238		
36000.0		152		
343230.0		95		
138942.32		198		
218825.2		87		
202600.0		141		
65312.4		204		
228000.0		96		
40826.52		165		
36127.62		234		
185100.0		None		
195663.16		88		
39019.78		53		
411211.15		240		
146191.71		197		
140314.72		131		
122247.76		100		


22. Рассчитать средний чек клиенту по выбранному менеджеру

In [8]:
q("""SELECT top(10) avg(salesRub), salesManager.surname
from sales
inner join salesManager on (salesManager.salesManagerId = sales.salesManagerId)
group by sales.salesManagerId, salesManager.surname
""")

1926.749248366013		Аброскина		
1094.02		Андреева		
3219.909090909091		Антонов		
7175.507796610177		БОТВИЧ		
20129.052857142855		Бакуров		
19984.735		Баранов		
2612.605204301075		Баранов		
4378.140936675322		Березина		
9753.485976027421		Валов		
3729.434210526316		Васильев		


23. Рассчитать средний чек клиента по филиалу

In [9]:
q("""SELECT avg(salesRub), branchName
from sales
inner join branch on (branch.branchId = sales.branchId)
group by branch.branchId, branchName
""")

4584.415962620793		Екатеринбург		
2142.0263963320244		ИнтернетМагазин		
7309.865479662998		Москва		
5978.902595652673		Новосибирск		
14346.35732418522		Пятигорск		
4976.33986232961		Самара		


24. Средний чек клиента по менеджерам внутри филиалов

In [10]:
# TODO 

25

In [11]:
# TODO 

26

In [12]:
# TODO 

27. Рассчитать АВС товарных позиций

In [13]:
# TODO 

28\. -

29\. -

30\. -

31\. -

32. Есть менеджеры, которые продают в более чем одном филиале, найти их!

In [14]:
q("""--sql
with sms(managerId, numberOfBranches)
as
(
    select sm.salesManagerId, count(distinct branch.branchId)
    from sales
    inner join branch on (sales.branchId = branch.branchId)
    inner join salesManager as sm on (sales.salesManagerId = sm.salesManagerId)
    group by sm.salesManagerId
    having count(distinct branch.branchId) > 1
)

select managerId, numberOfBranches, salesManager.surname, salesManager.[names]
from sms
inner join salesManager on (sms.managerId = salesManager.salesManagerId)
""")

7		2		Березина		Стелла		
9		2		Валов		Игорь		
10		2		Васильев		Дмитрий		
13		2		Воробьева		Юлия		
19		2		Долганов		Геннадий		
20		2		Дудкина		Татьяна		
28		2		Исакова		Евгения		
31		2		Карпухина		Елена Андреевна		
34		2		Ковалевская		Алёна		
36		2		Комаров		Эдуард		
37		2		Комарова		Василина		
47		2		Мальшин		Олег		
53		2		Меньшиков		Сергей		
54		2		Минеева		Ксения		
67		3		Рензяк		Диана		
74		2		Семенов		Олег		
76		2		Собакин		Денис		
78		3		Ставицкая		Анастасия		
83		2		Хакунов		Анзор		
85		2		Чуканова		Инна		
87		2		Шашкина		Полина		
89		2		Ширяев		Александр		


33. Найти сумму продаж по менеджерам, со следующими условиями. Какими условиями?

34. Выгрузки с датами, различными. The fuck?

35. Показать, что среднее кол-во чеков и среднее по транзакции это разные вещи.

In [15]:
# todo

36\. Найти количество уникальных счетов при условиях. Каких условиях?